<a href="https://colab.research.google.com/github/dnguyend/EmbeddedGeometry/blob/main/rbrown/colab/DemoVariableSphere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\newcommand{\R}{\mathbb{R}}$
# Testing the variable sphere class
* This one tests the variable_sphere class.
* This class consider a sphere $S_V^d$ in $\R^{d+1}$ with radius $\R$, but with the metric given by
$\langle \xi, \eta_x = \xi^T(D(x)\odot \eta) + \epsilon (C(x)^T\xi)(C(x)^T\eta)$
where $D$ and $C$ are two vector-valued functions, and $\epsilon\in \{\pm 1\}$. We will  assume $D(x)$ has all positive entries while $C(x)$ could have negative entries, however the over all metric is positive definite as a bilinear pairing on $\R^{d+1}$. Obviously if $\epsilon = 1$ and $C$ has nonnegative entries then this is satisfied.
* This class includes thhe ellipsoids metric, it is complicated  enough to hopefully test the algorithm for robustness, yet the projection and Levi-Civita  connection could be computed efficiently.


# Credential
If you want to run the code, since the repository is private you  need to load with your Github credential. You can use your common credential, the sign in code is transparent here (We replace @ by %40 in email address in  the code below). You can also create a  [special token](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens) for this repository, so if that token is stolen by anybody, the best he/she can do is go in to this repository, but cannot do harm to anything else.

Some other ways to load private repository are [here](https://stackoverflow.com/questions/48350226/methods-for-using-git-with-google-colab) if  you don't like the widget below.

In [1]:
import ipywidgets as widgets
from IPython.display import display
import subprocess


class credentials_input():
    """To access a private repository
    Include this snippet of codes to colab if you want to access
    a private repository
    """
    def __init__(self, repo_name):
        self.repo_name = repo_name
        self.username = widgets.Text(description='Username', value='')
        self.pwd = widgets.Password(
            description='Password', placeholder='password here')

        self.username.on_submit(self.handle_submit_username)
        self.pwd.on_submit(self.handle_submit_pwd)
        display("Use %40 for @ in email address:")
        display(self.username)

    def handle_submit_username(self, text):
        display(self.pwd)

    def handle_submit_pwd(self, text):
        username = self.username.value.replace('@', '%40')
        cmd = f'git clone https://{username}:{self.pwd.value}@{self.repo_name}'
        process = subprocess.Popen(
            cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        output, error = process.communicate()
        print(output, error)
        self.username.value, self.pwd.value = '', ''



In [3]:
credentials_input('github.com/dnguyend/GeometricML.git')

'Use %40 for @ in email address:'

Text(value='', description='Username')

Password(description='Password', placeholder='password here')

b'' b"Cloning into 'GeometricML'...\n"


If there is a mistake, please  clear  your output, otherwise your credential may be displayed. Or you could change the handle_submit_pwd above to NOT displaying the  output/error.

In [4]:
import jax
import jax.numpy as jnp
import jax.numpy.linalg as jla
import jax.scipy.integrate as jsi
from jax import random, jvp

import numpy as np

import GeometricML.rbrown.simulation.heat_kernels as hkm
import GeometricML.rbrown.manifolds.variable_sphere as vpm
from GeometricML.rbrown.utils.utils import (grand, get_complement_basis)
jax.config.update("jax_enable_x64", True)

$\newcommand{\rD}{\mathrm{D}}$
Creating a sphere  of  radius R with a variable metric.
In this case $D(x) =  v_{D0} + v_{D1}\odot x^{\odot 2}$, $C(x) = v_C\odot x$, we also supply Jacobian of $D$ and $C$, and the cross terms $x_D, x_C$ so that:
$$\chi_D^T(\omega_1, \omega_2)\omega_0 = \rD_{\omega_0} \{(D(x)\odot\omega_1)^T\omega_2\}\\
\chi_C^T(\omega_1, \omega_2)\omega_0 = \rD_{\omega_0} \{(C(x)\odot\omega_1)^T\omega_2\}
$$
for the calculation of the Christoffel  function.

In [5]:
from jax import random, jvp
key = random.PRNGKey(0)

d = 3
R = 1.3

vD0, key = grand(key, (d+1,))
vD0 = vD0*vD0 + 1
vD1, key = grand(key, (d+1,))
vD1 = vD1*vD1
vC, key = grand(key, (d+1,))

sph = vpm.VariableSphere(R, d,
                      lambda x: vD0 + vD1*x**2,
                      lambda x: vC*x,
                      lambda x, omg: 2*vD1*x*omg,
                      lambda x, omg: vC*omg,
                      lambda x, omg1, omg2: 2*vD1*x*omg1*omg2,
                      lambda x: vC,
                      1)


Check the projection is metric invariant, and various compatiblity conditions

In [6]:
x, key = sph.randpoint(key)
print(jnp.sum(x*x) - R**2)
y, key = sph.randpoint(key)

v, key = sph.randvec(key, x)
print(jnp.sum(x*v))

va, key = sph.randvec(key, x)
vb, key = sph.randvec(key, x)
omg, key = sph.rand_amb(key)
omg0, key = sph.rand_amb(key)


omg1 = sph.metric(x, omg)
omg2 = sph.inv_metric(x, omg1)
print("test inv_metric is invert of metric")
print(omg2 - omg)
print(jnp.sum(va*omg1) - sph.inner(x, va, omg))

pomg = sph.proj(x, omg)
print("test projection is metric compatible")
print(jnp.sum(x*pomg))
print(sph.inner(x, omg, va) - sph.inner(x, sph.proj(x, omg), va))

print("test derivative formula for metric")
print(jvp(lambda x: sph.metric(x, omg1), (x,), (omg0,))[1])
print(sph.d_metric(x, omg0, omg1))

print("test the index raising formula")
print(jnp.sum(sph.d_metric(x, omg0, omg1)*omg2))
print(jnp.sum(sph.x_metric(x, omg1, omg2)*omg0))

print("test the derivative of the projection")
print(jvp(lambda x: sph.proj(x, vb), (x,), (va,))[1])
print(sph.d_proj(x, va, vb))



4.440892098500626e-16
3.3306690738754696e-16
test inv_metric is invert of metric
[ 0.00000000e+00  6.66133815e-16 -1.11022302e-16 -8.32667268e-17]
1.1102230246251565e-16
test projection is metric compatible
2.914335439641036e-16
1.1102230246251565e-15
test derivative formula for metric
[ 3.25507442 -1.17230552 -5.89113721  0.31270068]
[ 3.25507442 -1.17230552 -5.89113721  0.31270068]
test the index raising formula
-3.201881617282374
-3.201881617282371
test the derivative of the projection
[-1.17332502  2.14297304 -0.62404931  0.9826862 ]
[-1.17332502  2.14297304 -0.62404931  0.9826862 ]


* Test the connection is indeed the Levi-Civita connection

In [7]:
print(jvp(lambda x: sph.inner(x, vb, vb), (x,), (va,))[1])
print(2*sph.inner(x, vb,
                  jvp(lambda x: sph.proj(x, vb), (x,), (va,))[1] + sph.gamma(x, va, vb)))


-9.221260773940756
-9.221260773940761


* test generating the orthogonal basis. The usual method, null space of the constraint equation is usually expensive. In this case, since parallel transport for the round metric is cheap, we  use it to create an (not orthogonal) basis, then normalize it to make it orthogonal

In [8]:
b1 = sph.make_tangent_basis(x)
print(b1.shape)

print(
    jnp.array(
        jax.vmap(lambda a:
      [sph.inner(x, a, b1[:, i]) for i in range(b1.shape[1])], in_axes=1)(b1)).reshape(b1.shape[1], -1))



(4, 3)
[[ 1.00000000e+00 -1.56125113e-16  1.24900090e-16]
 [-1.83880688e-16  1.00000000e+00 -3.50847823e-16]
 [ 1.24900090e-16 -3.45643653e-16  1.00000000e+00]]


* check the function make_ambient_basis giving us an orthogonal basis for the metric defined by $D(x), C(x)$

In [9]:

b2 = sph.make_ambient_basis(x)
print(b2.shape)
print(
    jnp.array(
        jax.vmap(lambda a:
      [sph.inner(x, a, b2[:, i]) for i in range(b2.shape[1])], in_axes=1)(b2)).reshape(b2.shape[1], -1))


(4, 4)
[[ 1.00000000e+00  1.38777878e-17  1.64798730e-17  1.56125113e-17]
 [ 1.38777878e-17  1.00000000e+00  1.38777878e-17 -1.66533454e-16]
 [ 1.64798730e-17  0.00000000e+00  1.00000000e+00 -2.18575158e-16]
 [ 1.64798730e-17 -1.80411242e-16 -2.18575158e-16  1.00000000e+00]]


# The Spherical Laplacian and the Brownian drift on the variable metric on the EUclidean space.

* The Laplacian is
$$\Delta f = g^{mn}(\partial^2_{\xi^m\xi^n}-\Gamma^l_{mn}\partial_{\xi^l})f
$$

The Brownian drift is
$-\frac{1}{2}g^{mn}\Gamma^l_{mn}\partial_{\xi^l}$.


First check the g matrix.

In [10]:
imat = jnp.eye(d+1)
gmat = jnp.array(
        jax.vmap(lambda a:
        [sph.inner(x, a, imat[:, i]) for i in range(imat.shape[1])], in_axes=1)(imat)).reshape(imat.shape[1], -1)

print(gmat)
print(jnp.diag(sph.D(x)) + sph.pm*sph.C(x)[:, None]@sph.C(x)[None, :])


[[ 1.09446095  0.13948704 -0.01677233 -0.0269987 ]
 [ 0.13948704  3.26489484 -0.23693234 -0.38139394]
 [-0.01677233 -0.23693234  1.18213097  0.04585992]
 [-0.0269987  -0.38139394  0.04585992  1.82617129]]
[[ 1.09446095  0.13948704 -0.01677233 -0.0269987 ]
 [ 0.13948704  3.26489484 -0.23693234 -0.38139394]
 [-0.01677233 -0.23693234  1.18213097  0.04585992]
 [-0.0269987  -0.38139394  0.04585992  1.82617129]]


In [13]:
from jax import jvp, grad

def LaplacianAmbient_0(self, x, f):
  tup = jnp.zeros(self.d+1)
  ret = 0
  for i in range(self.d+1):
    v_i = tup.at[i].set(1.)
    d2 = jvp(lambda x: grad(f)(x), (x,), (v_i,))
    ret += jnp.sum((d2[1]- self.gamma_ambient(x, v_i, d2[0]))*self.inv_metric(x, v_i))
  return ret

def LaplacianAmbient(self, x, f):
  tup = jnp.zeros(self.d+1)
  ret = 0
  grad_jit = jax.jit(grad(f))
  gmat = jnp.diag(sph.D(x)) + sph.pm*sph.C(x)[:, None]@sph.C(x)[None, :]
  igmat = jla.inv(gmat)

  hess = jax.vmap(lambda xi: jvp(grad_jit, (x,), (xi,))[1])(jnp.eye(self.d+1))
  # print(hess)
  # print(igmat)
  # print(jnp.sum(hess*igmat))
  gm = 0
  grdx = grad_jit(x)
  for i in range(self.d+1):
    v_i = tup.at[i].set(1.)
    for j in range(self.d+1):
      v_j = tup.at[j].set(1.)
      gm += igmat[i, j]*jnp.sum(gamma_ambient(self,  x, v_i, v_j)*grdx)

  return jnp.sum(hess*igmat) - gm


print(LaplacianAmbient(sph, x, lambda x: jnp.sum(x*x)))
print(sph.LaplacianAmbient(x, lambda x: jnp.sum(x*x)))


5.445489629333139
5.445489629333139


In [15]:

def bd2(self, x):
  tup = jnp.zeros(self.d+1)
  ret2 = 0
  for i in range(self.d+1):
    v_i = tup.at[i].set(1)
    for j in range(self.d+1):
      v_j = tup.at[j].set(1)
      ret2 += jnp.sum(v_i*self.inv_metric(x, v_j))*sph.gamma_ambient(x, v_i, v_j)
  return -0.5*ret2

ret2 = bd2(sph, x)
print(ret2)
print(sph.brownian_drift_ambient(x))


[ 0.18521857  0.05386831 -0.0065454  -0.00939676]
[ 0.18521857  0.05386831 -0.0065454  -0.00939676]
